Here, we get relevant data and parse a html converted from a pdf found at the url: of the top chinese characters:   

http://www.zein.se/patrick/3000char.html

In [2]:
import requests
from bs4 import BeautifulSoup
import string
import csv
import re
import pandas as pd
import hanzidentifier
import unicodedata


# Suppress just SettingWithCopyWarningda
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

### Part 1: get initial data of simpilifed and traditional characters, along with phrases/multi-character words in traditional

Loop through the tables of the html. With the exception of the first one, they follow a predictable pattern of definition, words from by multiple characters. 

We create an initial datasets: one with characters only (both simplifed and traditional), together another with multiple characters and their pronunciations. The character category(cat) is the index of the character (used for ranking relatively difficulty).

For alternative characters (mainly numbers) we only show it for the single character word

Special care is used to ensure merged traditional forms can be separated based on the word and meaning, such as (幹乾=>干)

In [ ]:
#our key helper function to parse definitions and pronunciations apart given the raw main character form, etc
#returns a list of lists with the order ["cat", "word/character", "pronunciation", "definition", "code","alt"]. Code is if the word uses traditional/simplified (s,t,a). More contains extra information (in few cases where there is an alternative character)
# since many words may have compound simplified characters in this list, we will later need to convert again for the words that 
#note pronunciation is the the pronunciation of the character in the word
# are desired to only be written in traditional.
#Therefore, each row will be written twice, one for traditional one for simplified
#for single characters, we use the first pronunciation listed

def getbetween(input, start, end): #helper function to get the string in between two symbols (the first matching abtter)
    startind = input.find(start)
    #if the first one is not found, return empty:
    if startind == -1: 
        return ""
    #narrow down input
    input = input[startind+len(start):]
    endind = input.find(end)
    #if the first one is not found (but start is found), return the string starting from the end 
    if endind == -1:
        return input
    return input[:endind].strip() #otherwise, return 

#current helper function to get all text between characters (a list). Add space to the list, to not include whitespace
def getall_betweenchars(charlist, text):
    #if the length of the list is empty
    if len(charlist) == 0:
        return "" #empty string
    if len(charlist) == 1: #if there is only element, return the only one
        return charlist[0] #empty string
    #get the max and lowest indices 
    charlist.append(" ")
    max = 0 
    max_string = "" #string with the min index
    min = 1000000 #absurdly large
    min_string = "" 
    for char in charlist:
        #find the index of the charcter in the, text
        ind = text.find(char)
        if ind > max:
            max = ind
            max_string = char
        if ind < min:
            min = ind
            min_string = char
    l2 = len(max_string)
    return text[min:max+l2]

def get_wordList(main, des, cat):
    ret = [] #list to return

    simp = main.split("(")[0] #get the simplified form
    alt = getbetween(main,"(A",")") #set as empty string initially for the alternative form #check for alternative form, by default if there is non, this function should return an empty string 
    #the the traditional form, if any 
    trad = getbetween(main,"(F",")")
    if trad == "": #if function is empty, get to be the same as the simplified character
        trad = simp
    print("current simp. :"+simp+", current trad. :"+trad)
    frags = des.split("[") #split on [
    #iterate through diffenent fragments with pronunciations and descriptions within the definition
    #count the number of total definitions
    i = 0
    for frag in frags:
        #get the separate pronunciations
        pro = frag.split("]")[0]
        print("current pronunciation:"+pro)
        #if not empty
        if len(pro) > 0:
            #get rid of the part that is not part of the pronunciaiton
            defis = frag.replace(f"{pro}]", "")
            #print("pronunciation:", pro, ",definitions:", defis)
            defis = defis.split(";")#get separate defintions
            defis = [x for x in defis if x.strip()] #remove empty strings
#replace empty strings
            for d in range(0,len(defis)):
                defi =  defis[d].strip() #get a certain definition, with leading/ training whitespace remove. def is the simplified def. 
                print("current def:", defi)
                
                newTrad = False #store if at least 1 new trad. character is found
                #check for potentially multiple traditional characters
                def_parts = defi.split(", ")
                exclude = [] #append for parts to exclude in the traditional definitions (since we have to substract the separate traditional definition in this case)
                for part in def_parts:
                    print("current part:", part)
                    trad2 = getbetween(part,"(F",")") #modify the fragment string
                    if len(trad2) != 0: #if function is not empty, get to be the same as the simplified character. In this case we have a in line traditional character
                        newTrad = True
                        minidef = getbetween(part ,trad2, ", ")
                        minidef = minidef.split(")")[1].strip() #get the minidef (specialized definition between the new character and commas and if any)
                        nestchar = trad2.split(",")#for the few cases where there are multiple nested characters like (裏,裡), we set the 2nd one to be the variant
                        mainchar = nestchar[0]
                        #new_alternative character
                        new_alt = ""
                        print("minidef:", minidef, ", in def:", defi, "with character:", mainchar, ", and alternative:", new_alt)
                        if len(nestchar) > 1: new_alt = nestchar[1] #assign the variant to be the 2nd one
                        #in the case we have a subdef, we append an extra traditional entry 
                        new1 = [cat, mainchar, pro, minidef, "t",new_alt] #"cat", "word/charcter", "pronunciation", "definition", "code","alt"
                        ret.append(new1)
                    
                        ex_str = f", (F {trad2}) {minidef}"
                        exclude.append(ex_str) #string to replace in the original definition for the traditional definition
                traddef = defi  #create new traditional definition excluding the existing one
                print("exclusion list:", exclude)
                for ex in exclude:
                    traddef = traddef.replace(ex, "")

                
                chinese_pattern  = re.compile(r'[\u4E00-\u9FFF]+') #regex to get list of matching characters 
                match = chinese_pattern.findall(traddef)
                han = getall_betweenchars(match, traddef)   #helper to get all chars between the list of matching characters 
                print("found subcharacter:"+han+"|")
                if len(han)>1 and i!=0: #if the def definition contains multiple chinese characters, we do extra processing to extract the characters, and if this is not a new trad 
                    charind = defi.find(han) #find last index
                    char = han.strip() # the character for the word 
                    defi = defi[charind+len(han):] #modify the definition to be the remaining part
                    new1 = [cat, char, pro, defi, "s",alt] #"cat", "word/charcter", "pronunciation", "definition", "code","alt"
                    new2 = [cat, char, pro, traddef, "t",alt] #for now
                    ret.append(new1)
                    ret.append(new2)
                    #in this case, we append as so 
                    print("ran if"+str(new2))
                elif newTrad==False: #assume that there is only one character to the definition, and no traditional characters found
                    new1 = [cat, simp, pro, defi, "s",alt] #"cat", "word/charcter", "pronunciation", "definition", "code","alt"
                    new2 = [cat, trad, pro, defi, "t",alt]
                    ret.append(new1)
                    ret.append(new2)
                    print("ran elif 1"+str(new2))
                else: 
                    print("ran else (diff traditional and simplified definition)")

                    new1 = [cat, simp, pro, defi, "s",alt] #simplified definotion is as usual"
                    new2 = [cat, trad, pro, traddef, "t",alt]
                    ret.append(new1)
                    ret.append(new2)
                #print("character:"+char+", definition:"+defi)
                i+= 1

    return ret 

In [ ]:
link = "./main_char_set-converted.html"

with open(link, 'r') as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')
tables = soup.find_all("table")  
grand_list = [] #list used to create the dataframe in the end
#count the number of tables 
index = 0;
for table in tables:
    rows = table.find_all("tr") #extract all tables
    for row in rows: 
        # Extract all cells (td or th)
        cols = row.find_all(["td"]) #print the number of cols (exclude the header) 
        #valid character columns have 3 columns 
        if len(cols) == 3:
            #get the number column 
            col1 = cols[0].get_text()
            #get the char column 
            col2 = cols[1].get_text()
            #get description column, this is complicated and is where we split the characters from the words, using a helper function
            col3 = cols[2].get_text()
            #call our key helper function that retruns a list to be appended to the grand_list 
            res = get_wordList(col2, col3, cat=index)
            grand_list.append(res)
            index += 1
print("total number of characters listed:", index)

In [ ]:
print(len(grand_list))
grand_list

As expected, the list has 2431 subcomponents, but to effectively create the dataframe we need to remove a layer. 

In [ ]:
master_list = []
for l1 in grand_list:
    for l2 in l1:
        print(l2)
        master_list.append(l2)

Saving to a new csv file. First, remove the last two and first two entries from the, its clean its garbage

In [ ]:
master_list = master_list[2:len(master_list)-2]
for l in master_list:
    print(l)

We now save to a dataframe 

In [ ]:
data =  pd.DataFrame(master_list, columns=["cat", "word/character", "pronunciation", "definition", "code","alt"])
data.to_csv("main_list.csv", index=False)


### Part 2: Quality repairs

Creating a map of simplified to traditional characters so we can effectively edit multicharacter words here. Note that this is map from simplified characters to a list of traditional ones.

In [ ]:
#isolate all rows of single characters 
single_characters = data[data["word/character"].str.len() == 1]
print(len(single_characters))
single_characters.head(n=1)
#save 
single_characters.to_csv("character_list.csv", index=False)
#iteration based on category
cats = single_characters["cat"].unique()
StoTmap = {} #character map

for cat in cats:
    subset =  single_characters[single_characters["cat"] == cat]
    subset = subset.drop_duplicates(subset=['word/character']) #get the unique character rows 
    char_list = set(subset["word/character"]) #get the of characters
    #the simplified character is the one where the code is "s"
    simp = subset[subset["code"]=="s"].iloc[0]["word/character"]
    trad = list(subset[subset["code"]=="t"]["word/character"]) #get list of purely traditional ch
    if len(trad) == 0: #if the list of traditional characters is empty, then they must be the same
        StoTmap[simp] = [simp]
    else:
        StoTmap[simp] = list(trad)
for key in StoTmap:
    print(key,",", StoTmap[key])

get the inverse map. this is easy since each traditional character maps to just one simplified one. 

In [ ]:
TtoS = {}
for key in StoTmap:
    Tlist = StoTmap[key]
    for Tchar in Tlist:
        TtoS[Tchar] = key 
for key in TtoS:
    print(key,",", StoTmap[key])

Now, attempt to convert most compound words and definition characters to traditional automatically using this dictionary (for cases where the category is traditional). For cases of multiple mappings, we skip and print out a message highlighting the index to correct.

In [ ]:
charmap = StoTmap
#store failed indices for the word/character column
failed_chars = []
#store failed indices for the definition column
definition_chars = []
def convert(data): #    
    for index, row in data.iterrows():
        #first convert characters 
        code = row["code"]
        word = row["word/character"]
        defi = row["definition"]
        #check if variables are strings or not
        if (isinstance(defi, str) == False):
            print(f"definition at index {index} is not a string: {defi}, for char {word}")
            print("converted to:", str(defi))

        if code == "t":
            #convert the word first 
            new_word = "" #new string 
            for char in str(word):
                #check if in dict 
                if char in charmap:
                    char_list = charmap[char]
                    if (len(char_list)>1): 
                        print("failed to word/character column at index:", index,", with char:", char, "failing", ", context is:", word)
                        failed_chars.append(index)
                        new_word += char #assign back to old word in this case
                    else: 
                        new_word += char_list[0] #increment 
                else:
                    new_word += char #increment like usual 
            data.at[index,"word/character"] = new_word #assign 

            #then, convert the description characters. use the same character if possible
            new_defi = "" #new string 
            for char in str(defi):
                #check if in dict 
                if char in charmap:
                    char_list = charmap[char]
                    if (len(char_list)>1): 
                        print("failed to definition column at index:", index,", with char:", char, "failing", ", context is:", word)
                        definition_chars.append(index)
                        new_defi += char #increment like usual 
                    else: 
                        new_defi += char_list[0] #increment 
                else:
                    new_defi += char #increment like usual 
            data.at[index,"definition"] = new_defi #assign 

    return data
data = convert(data)
data.to_csv("./main_list.csv", index=True)

We then create a function to spot traditiona words and definitions that are not traditional characters (meaning one character that is in the simplified map cannot be easily converted because of the multiple-mapping issue)

In [ ]:
def check(data): #    
    for index, row in data.iterrows():
        #first convert characters 
        code = row["code"]
        word = row["word/character"]
        defi = row["definition"]

        if code == "t":
            #convert the word first 
            for char in str(word):
                #check key if in dict 
                if char in charmap:
                    char_list = charmap[char]
                    if (len(char_list)>1): 
                        print("word/character column misses a traditional character at:", index,", char:", char, f", possibly mapping to: f{char_list}") 
                        print(f"\t context is:", word)
                        failed_chars.append(index)
            for char in str(defi):
                #check if in dict 
                if char in charmap:
                    char_list = charmap[char]
                    if (len(char_list)>1): 
                        print("definition column misses a traditional character at:", index,", char:", char, f", possibly mapping to: f{char_list}") 
                        print(f"\t context is:", word)
                        definition_chars.append(index)

    return

In [ ]:
data = pd.read_csv("./main_list.csv")
check(data)

Print out failed indices text for word/character column. From the outputs it is clear that the all failures are a single character, and so far words do not have their actual pronunciation, we aim to fix that here


Changes were manually done based on the output. We save to a new file name, main_final_list and read again and reindex.

Many changes were made to delete duplicate entries caused by inconsistent formating ,etc

### Part 3: final dataset export, more quality changes

We now do more quality changed. re run this part on the new data until its fixed

First, get a new character map and match based on the changes. note now we dont drop based on both the character and pronunciation list

In [ ]:
path = "./main_final_list.csv"
new_data = pd.read_csv(path)

# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = new_data.columns[new_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
new_data.drop(columns=unnamed_cols, inplace=True)
new_data = new_data.reindex()
new_data.to_csv(path, index=True)

In [ ]:
#isolate all rows of single characters 
single_characters = new_data[new_data["word/character"].str.len() == 1]
print(len(single_characters))
single_characters.head(n=1)
#save 
single_characters.to_csv("character_list.csv", index=False)
#iteration based on category
cats = single_characters["cat"].unique()
StoTmap = {} #character map

for cat in cats:
    #get all characters 
    subset = single_characters[single_characters["cat"] == cat]
    #get the traditional subset first
    trad_subset =  subset[subset["code"] == "t"]
    simp_subset =  subset[subset["code"] == "s"]
    trad_subset = trad_subset.drop_duplicates(subset=['word/character']) #get the unique character rows 
    simp_subset = simp_subset.drop_duplicates(subset=['word/character'])
    
    trads = list(set(trad_subset["word/character"]))
    simps = list(simp_subset["word/character"])
    simp = simps[0]
    
    StoTmap[simp] = list(trads)
#print the map 
for key in StoTmap:
    listc =  StoTmap[key]
    print(key,",", StoTmap[key])
    if (len(listc) == 0):
        print("empty trad list for simp. character:", key)

Finding the inverse map

In [ ]:
TtoS = {}
for key in StoTmap:
    Tlist = StoTmap[key]
    for Tchar in Tlist:
        TtoS[Tchar] = key 
for key in TtoS:
    print(key,",", TtoS[key])

Note: checking for empty lists, we did further corrections to the main_final_list. The 5 Errors appear to be from the alternate characters not properly parsed. We then ran this part again

Check the data again using the same function but the new character map

In [ ]:
charmap = StoTmap
#store failed indices for the word/character column
failed_chars = []
#store failed indices for the definition column
definition_chars = []
def check(data): #    
    for index, row in data.iterrows():
        #first convert characters 
        code = row["code"]
        word = row["word/character"]
        defi = row["definition"]

        if code == "t":
            #convert the word first 
            for char in str(word):
                #check key if in dict 
                if char in charmap:
                    char_list = charmap[char]
                    if (len(char_list)>1): 
                        print("word/character column misses a traditional character at:", index,", char:", char, f", possibly mapping to: f{char_list}") 
                        print(f"\t context is:", word)
                        failed_chars.append(index)
            for char in str(defi):
                #check if in dict 
                if char in charmap:
                    char_list = charmap[char]
                    if (len(char_list)>1): 
                        print("definition column misses a traditional character at:", index,", char:", char, f", possibly mapping to: f{char_list}") 
                        print(f"\t context is:", word)
                        definition_chars.append(index)

    return
check(new_data)


finally, view rows with invalid characters, and manually repair altter (F: with a more clear traditional mark) and (S with a more clear simplified mark). Check for duplicate rows with the same pronunciation and character [noting many characters can have different pronuciation] (within the same category), then manually fix 

In [ ]:
#create function check if a string of chracters is all chinese
def contains_chinese(text): 
    pattern = r'[\u4e00-\u9fff\u3400-\u4dbf]'  
    return bool(re.search(pattern, text))

#create function check if a single characer is all chinese or punctuation
def char_chinese_or_punctuation(text): 
    if len(text) < 1:
        print("must be a single string character")
        return False
    pattern = r'[\u4e00-\u9fff\u3400-\u4dbf]' 
    isChinese = bool(re.search(pattern, text))
    isPronuncation = text in string.punctuation
    return (isChinese or isPronuncation)

def allChinese(text):
    for char in text: 
        if char_chinese_or_punctuation(char) == False: #use function to check if a single characer is chiense
            return False 
    return True
print(allChinese("领导"))
print(allChinese("元宵,...节"))
print(allChinese("地支 dìzhī the Twelve Terrestrial Branches"))

#helper function, that find the help of a map from Traditional character in a string to simplified, attempts conversion of a string
def toSimplified(text, map):
    rettext = "" 
    #keep track of string index
    i = 0
    for char in str(text):
        #if the chracter is chiense 
        if contains_chinese(char) == True:
            #check if the character is in the map
            if char in map:
                rettext += map[char]
            else: #print that the character could not be converted
                print(f"trad char {char} cannot be converted based our map, using the same character as the original text")
                rettext += text[i]
        else: 
            rettext += text[i]
        i+=1#in this case we do not change =
    return rettext

#helper function, that find the help of our custom map a map from simplified characters to traditional, attempts conversion of a string
#handles cases where there are multiple mappings
def toTraditional(text, map):
    rettext = "" 
    #keep track of string index
    i = 0
    for char in str(text):
        #if the chracter is chiense 
        if contains_chinese(char) == True:
            #check if the character is in the map
            if char in map:
                #check the mapped list
                if len(map[char]) == 1:
                    rettext += map[char][0]
                else: 
                    print(f"simp char {char} has multiple possible traditional chars, use manual mapping, using the same character as the original text")
                    rettext += text[i] 
            else: #print that the character could not be converted
                print(f"simp char {char} cannot be converted based our map, using the same character as the original text")
                rettext += text[i]
        else: 
            rettext += text[i]
        i+=1#in this case we do not change =
    return rettext

Checking function to convert appropriately to traditional or simplified, and running function appropriately, based on the output, we edit the file again, if conversion is not possible. We first check if we need to convert using a special python library. note that often times, it is fine if the definition of a simplified word/character contains traditional since it is giving extra information. However, all such errors in the word must be corrected

In [ ]:
#testing functions
print(toSimplified("hi我不知道你在哪裏，在台灣嗎!", TtoS))
print(toTraditional("hi我不知道你在哪里在台湾吗!", StoTmap))

In [ ]:
for index, row in new_data.iterrows():
    #first convert characters 
    code = str(row["code"])
    word = str(row["word/character"])
    defi = str(row["definition"])
    
    new_word = word
    new_def = defi
    for char in word:
        if code == "s" and contains_chinese(char): #run checks on all inerated 
            if hanzidentifier.is_simplified(char) == False:
                print("error in word at index:", index)
                new_word = toSimplified(word, TtoS)
        if code == "t" and contains_chinese(char):
            if hanzidentifier.is_traditional(char) == False:
                print("error in word at index:", index)
                new_word = toTraditional(word, StoTmap)
    for char in defi:
        if code == "s" and contains_chinese(char):
            if hanzidentifier.is_simplified(char) == False:
                print("error in def at index:", index)
                new_def = toSimplified(defi, TtoS)
        if code == "t" and contains_chinese(char):
            if hanzidentifier.is_traditional(char) == False:
                print("error in def at index:", index)
                new_def = toTraditional(defi, StoTmap)
    #set with the new word if needed 
    #with with the new def if needed
    if new_def != defi:
        new_data.at[index, "definition"] = new_def
        print("changed definition at:", index)
    if new_word != word: 
        new_data.at[index, "word/character"] = new_word
        print("changed word/character at:", index)

In [ ]:
for index, row in new_data.iterrows():
    #first convert characters 
    code = str(row["code"])
    word = str(row["word/character"])
    defi = str(row["definition"])
    
    for char in word:
        if code == "s" and contains_chinese(char): #run checks on all inerated 
            if hanzidentifier.is_simplified(char) == False:
                print(f"at word: {word} ({index})|{char} not simplified chinese")
        if code == "t" and contains_chinese(char):
            if hanzidentifier.is_traditional(char) == False:
                print(f"at word: {word} ({index})|{char} not traditional chinese")
    for char in defi:
        if code == "s" and contains_chinese(char):
            if hanzidentifier.is_simplified(char) == False:
                print(f"at def: {defi} ({index})|{char} not simplified chinese")
        if code == "t" and contains_chinese(char):
            if hanzidentifier.is_traditional(char) == False:
                print(f"at def: {defi} ({index})|{char} not traditional chinese")


Running until there is no output. indicating the problem has been fixed

In [ ]:
for index, row in new_data.iterrows():
    #first convert characters 
    code = row["code"]
    word = row["word/character"]
    defi = row["definition"]
    if allChinese(word) == False:
        print(f"word: {word}| at index: {index} is not all chinese")


Now, checking that definitions and words should be in the correct traditional or simplified format, checking using a special python library

In [ ]:
for index, row in new_data.iterrows():
    #first convert characters 
    code = str(row["code"])
    word = str(row["word/character"])
    defi = str(row["definition"])
    
    for char in word:
        if code == "s" and contains_chinese(char): #run checks on all inerated 
            if hanzidentifier.is_simplified(char) == False:
                print(f"at word: {word} ({index})|{char} not simplified chinese")
        if code == "t" and contains_chinese(char):
            if hanzidentifier.is_traditional(char) == False:
                print(f"at word: {word} ({index})|{char} not traditional chinese")
    for char in defi:
        if code == "s" and contains_chinese(char):
            if hanzidentifier.is_simplified(char) == False:
                print(f"at def: {defi} ({index})|{char} not simplified chinese")
        if code == "t" and contains_chinese(char):
            if hanzidentifier.is_traditional(char) == False:
                print(f"at def: {defi} ({index})|{char} not traditional chinese")


Now, replace (F, (T substrings in the definition column. done with find and replace

Saving after manual repair

In [ ]:
# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = new_data.columns[new_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
new_data.drop(columns=unnamed_cols, inplace=True)
new_data = new_data.reindex()
new_data.to_csv(path, index=True)

### Part 4: create specialized datasets for final export to json for a dataset of characters and words with unique characters

Importing helper functions. 

In [ ]:

def getbetween(input, start, end): #helper function to get the string in between two symbols (the first matching abtter)
    startind = input.find(start)
    #if the first one is not found, return empty:
    if startind == -1: 
        return ""
    #narrow down input
    input = input[startind+len(start):]
    endind = input.find(end)
    #if the first one is not found (but start is found), return the string starting from the end 
    if endind == -1:
        return ""
    return input[:endind] #otherwise, return 

#create function check if a string of chracters is all chinese
def contains_chinese(text): 
    pattern = r'[\u4e00-\u9fff\u3400-\u4dbf]'  
    return bool(re.search(pattern, text))

#create function check if a single characer is all chinese or punctuation
def char_chinese_or_punctuation(text): 
    if len(text) < 1:
        print("must be a single string character")
        return False
    pattern = r'[\u4e00-\u9fff\u3400-\u4dbf]' 
    isChinese = bool(re.search(pattern, text))
    isPronuncation = text in string.punctuation
    return (isChinese or isPronuncation)

def allChinese(text):
    for char in text: 
        if char_chinese_or_punctuation(char) == False: #use function to check if a single characer is chiense
            return False 
    return True

#helper function, that find the help of a map from Traditional character in a string to simplified, attempts conversion of a string
def toSimplified(text, map):
    rettext = "" 
    #keep track of string index
    i = 0
    for char in str(text):
        #if the chracter is chiense 
        if contains_chinese(char) == True:
            #check if the character is in the map
            if char in map:
                rettext += map[char]
            else: #print that the character could not be converted
                print(f"trad char {char} cannot be converted based our map, using the same character as the original text")
                rettext += text[i]
        else: 
            rettext += text[i]
        i+=1#in this case we do not change =
    return rettext

#helper function, that find the help of our custom map a map from simplified characters to traditional, attempts conversion of a string
#handles cases where there are multiple mappings
def toTraditional(text, map):
    rettext = "" 
    #keep track of string index
    i = 0
    for char in str(text):
        #if the chracter is chiense 
        if contains_chinese(char) == True:
            #check if the character is in the map
            if char in map:
                #check the mapped list
                if len(map[char]) == 1:
                    rettext += map[char][0]
                else: 
                    print(f"simp char {char} has multiple possible traditional chars, use manual mapping, using the same character as the original text")
                    rettext += text[i] 
            else: #print that the character could not be converted
                print(f"simp char {char} cannot be converted based our map, using the same character as the original text")
                rettext += text[i]
        else: 
            rettext += text[i]
        i+=1#in this case we do not change =
    return rettext

#helper function determine if a character has diacritic or not
def contains_diacritic(char):
  nfkd_form = unicodedata.normalize('NFKD', char)  # Normalize to decompose characters
  for x in nfkd_form:
    if unicodedata.combining(x):  # Check if the character is a combining character
      return True
  return False


checking new helper

In [ ]:
print(contains_diacritic("zhǎng"))
print(contains_diacritic("zhǎng"))
print(contains_diacritic("Equivalent"))
print(contains_diacritic("{Compare with 方"))


#### subpart 1:

Create our final character dataset (also includes multi-character words that have a single unique character)

In [ ]:
#get a reference for current unique single characters 
unique_characters = pd.read_csv("./character_list.csv")
unique_characters = unique_characters["word/character"].unique()
print(len(unique_characters))

In [ ]:
new_data = pd.read_csv("./main_final_list.csv")
cats = new_data["cat"].unique() #get the unique categories  
keep = [] #index of values to keep
#iterate based on category
for cat in cats:
    #get all characters 
    subset = new_data[new_data["cat"] == cat]
    subset = subset.drop_duplicates(subset=['word/character', "pronunciation", "code"]) #drop based on the same character, pronunciation, and code
    #iterate through the subset
    for index, row in subset.iterrows():
        #first convert characters 
        code = str(row["code"])
        word = str(row["word/character"])
        defi = str(row["definition"])

        #if the word is a single character, we keep this index
        if len(word) == 1:
            keep.append(index)
        else: #else interate through all characters (assuming it is one)
            for char in word: #if the character is chinese, and it is not in the set of chinese characters, we can keep these indices 
                if contains_chinese(char) == True and (char not in unique_characters):
                    keep.append(index)

#testable character dataset:
test_data = new_data.loc[keep] 
test_data = test_data.reindex()
print(len(test_data))
# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data.to_csv("./testable_characters_dataset.csv", index=True)


In [ ]:
#reindex again by reading in the same csv file 
test_link = "./testable_characters_dataset.csv"
test_data = pd.read_csv(test_link)
test_data = test_data.reindex()
print(len(test_data))
# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data.to_csv(test_link, index=True)

This dataset will be the set of characters and small words tested. For the compound words, we now attempt to get their pronunciation

In [ ]:
test_data = pd.read_csv("./testable_characters_dataset.csv")
test_data["full_pronunciation"] = "x" #add a column for the full pronunciation
for index, row in test_data.iterrows():
    #first convert characters 
    code = str(row["code"])
    word = str(row["word/character"])
    defi = str(row["definition"])
    charpro = str(row["pronunciation"])

    #if the word is a single character the actual pronuciation is trivially, this is the  actual one
    lw = len(word)
    if lw == 1:
        test_data.at[index, "full_pronunciation"] = charpro
    elif lw >= 2:
        print("compound word:", word)
        #iterate through the definition (it should be there). words by find the first and last char by diacritic
        #clear chinese characters in the definitions and trim
        for char in defi: 
            if contains_chinese(char): #find if the character is chinese 
                defi = defi.replace(char, "")
        defi = defi.strip()
        #split the definition by space
        defi_split = defi.split(" ")
        #the pronuciation should be the 1st thing
        full_pro =  defi_split[0]
        #save pronucniation
        test_data.at[index, "full_pronunciation"] = full_pro
# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data = test_data.reindex()
test_data.to_csv(test_link, index=True)
            

Checking manually to see if pronuciations for 2 character words are valid/filled, a few changes were made, but most were correct. We save to a new csv file (final_test_characters) to prevent accidental overwrite

In [ ]:
# test reading in 

test_data = pd.read_csv("final_test_characters.csv")
for index, row in test_data.iterrows(): 
    #first convert characters 
    code = str(row["code"])
    word = str(row["word/character"])
    defi = str(row["definition"])
    full_pro = str(row["full_pronunciation"])      
    if len(word) >= 2:
        print(f"found pronunciation {full_pro} for index: {index}, for word: {word}")#if we have a diacritic


Now, add a column without diacritics (this will be useful for one of the user's test versions)

In [ ]:
def remove_diacritics(input_str): #helper function to remove diacritcis
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

# Example usage:
text_with_diacritics = "Crème brûlée is delicious!"
text_without_diacritics = remove_diacritics(text_with_diacritics)
print(text_without_diacritics)

In [ ]:

test_data = pd.read_csv("final_test_characters.csv")
test_data["full_pronunciation_wo"] = "None"
for index, row in test_data.iterrows(): 
    full_pro = str(row["full_pronunciation"])      
    test_data.at[index, "full_pronunciation_wo"] = remove_diacritics(full_pro)

# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data = test_data.reindex()
test_data.to_csv("final_test_characters.csv", index=True)

### Part 5: Final stage for single characters

For now, we are done with painstaking manual changes to the characters data (final_test_characters.csv), and now get ready to export our data in a JSON format. First, however, we create a test version of our data

In [ ]:
path = "final_test_characters.csv" #"final_test_characters.csv"
test_data = pd.read_csv(path)
test_data["test_definition"]=None
for index, row in test_data.iterrows(): 
    defi = str(row["definition"])      
    pro = str(row["full_pronunciation"])   #get the pronuciation. Replace all instances with _ for the test definition
    rettext = "" #set to return
    #keep track of string index
    i = 0
    for char in defi:
        #if the chracter is chinese, we skip (essenially skipping it)
        if contains_chinese(char) == True:
            continue
        else:
            rettext += char
        #for each substring we build, we see if we can find any  {} enclosed text, and these other combinations
        start = "{"
        end = "}"
        found_enclosed = getbetween(rettext, start,end) #returns "" 
        rettext=rettext.replace(start+found_enclosed+end, "") #works even if nothing is found

        start = "(Equivalent:"
        end = ")"
        found_enclosed = getbetween(rettext, start,end) #returns "" 
        rettext=rettext.replace(start+found_enclosed+end, "") #works even if nothing is found

        start = "(Traditional:"
        end = ")"
        found_enclosed = getbetween(rettext, start,end) #returns "" 
        rettext=rettext.replace(start+found_enclosed+end, "")

        start = "(Simplified:"
        end = ")"
        found_enclosed = getbetween(rettext, start,end) #returns "" 
        rettext=rettext.replace(start+found_enclosed+end, "")

        i+=1#in this case we do not change =
    rettext = rettext.replace(pro, "_")
    test_data.at[index, "test_definition"] = rettext.strip() #change definition

# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data = test_data.reindex()
test_data.to_csv(path, index=True)

Check subset of data with empty test_definition

In [ ]:
subset = test_data[test_data["test_definition"] == ""]
print(len(subset))

In [ ]:
subset.head(n=36)

Drop these characters, then with no clear definition beside the extra infromation

In [ ]:
test_data = test_data[test_data["test_definition"] != ""]
# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data = test_data.reindex()
test_data.to_csv(path, index=True)
print(len(test_data))

reindex again

In [ ]:
path = "final_test_characters.csv" #"final_test_characters.csv"
test_data = pd.read_csv(path)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data = test_data.reindex()
test_data.to_csv(path, index=True)

Checking for rows where created rows test_definition or full_pronunciation_wo, full_pronunciation are missing. More manual repairs were done

In [ ]:
path = "final_test_characters.csv" #"final_test_characters.csv"
test_data = pd.read_csv(path, keep_default_na=False) #do not interpret nan pronunciation an null value

In [ ]:
subset = test_data[test_data["full_pronunciation"].isna()]
print(len(subset))
subset.head()

In [ ]:
subset = test_data[test_data["full_pronunciation_wo"].isna()]
print(len(subset))
subset.head(len(subset))

It appears all cases are because of nan characters with a nan pronunciation 

In [ ]:
subset = test_data[test_data["full_pronunciation"].isna()]
print(len(subset))
subset.head(len(subset))

Create our json (both a simplified and traditional version). This will be a dictionary of a list of dictionaries mapping to index

In [ ]:
import json

#### finally, save to JSON, dropping unnamed columns in the dataframe before converting

In [5]:
path = "final_test_characters.csv" #"final_test_characters.csv"
test_data = pd.read_csv(path, keep_default_na=False) #do not interpret nan pronunciation an null value
test_data.head()

,Unnamed: 0,cat,word/character,pronunciation,definition,code,alt,full_pronunciation,full_pronunciation_wo,test_definition
0,0,1,的,de,<grammatical particle marking genitive as well...,s,,de,de,<grammatical particle marking genitive as well...
1,1,1,的,de,<grammatical particle marking genitive as well...,t,,de,de,<grammatical particle marking genitive as well...
2,2,1,的,dí,"true, real",s,,dí,di,"true, real"
3,3,1,的,dí,"true, real",t,,dí,di,"true, real"
4,4,2,一,yī,"one, a little",s,壹,yī,yi,"one, a little"


In [6]:

test_data = test_data.loc[:, ~test_data.columns.str.startswith('Unnamed:')] #no unamed columns
test_data.to_csv(path)
test_data = pd.read_csv(path, index_col=0)
test_data.head()

,cat,word/character,pronunciation,definition,code,alt,full_pronunciation,full_pronunciation_wo,test_definition
0,1,的,de,<grammatical particle marking genitive as well...,s,NaN,de,de,<grammatical particle marking genitive as well...
1,1,的,de,<grammatical particle marking genitive as well...,t,NaN,de,de,<grammatical particle marking genitive as well...
2,1,的,dí,"true, real",s,NaN,dí,di,"true, real"
3,1,的,dí,"true, real",t,NaN,dí,di,"true, real"
4,2,一,yī,"one, a little",s,壹,yī,yi,"one, a little"


Start with traditional single characters first

In [ ]:
#start 
ourdict = {}
our_data = test_data[test_data["code"] == "t"]

json_index = our_data.to_json(orient='index')

# Print the JSON string
print(json_index)

This JSON is copied to the pages/data part of the webpage

Finally, get the same data for the simplified characters

In [ ]:
#start 
ourdict = {}
our_data = test_data[test_data["code"] == "s"]
cats = our_data["cat"].unique() #get the unique categories  
json_index = our_data.to_json(orient='index')

# Print the JSON string
print(json_index)

Manually replaced NaN string with null.

### Part 6: Final stage for words (multi-character words)- create the csv dataset of multi-character words (test_multi_list.csv) with test definitions to allow definitions of words to be tested. This csv is finally exported to JSON, in a format that is actually used by our aplication

Define helpers

In [ ]:

def getbetween(input, start, end): #helper function to get the string in between two symbols (the first matching abtter)
    startind = input.find(start)
    #if the first one is not found, return empty:
    if startind == -1: 
        return ""
    #narrow down input
    input = input[startind+len(start):]
    endind = input.find(end)
    #if the first one is not found (but start is found), return the string starting from the end 
    if endind == -1:
        return ""
    return input[:endind] #otherwise, return 

#create function check if a string of chracters is all chinese
def contains_chinese(text): 
    pattern = r'[\u4e00-\u9fff\u3400-\u4dbf]'  
    return bool(re.search(pattern, text))

#create function check if a single characer is all chinese or punctuation
def char_chinese_or_punctuation(text): 
    if len(text) < 1:
        print("must be a single string character")
        return False
    pattern = r'[\u4e00-\u9fff\u3400-\u4dbf]' 
    isChinese = bool(re.search(pattern, text))
    isPronuncation = text in string.punctuation
    return (isChinese or isPronuncation)

def allChinese(text):
    for char in text: 
        if char_chinese_or_punctuation(char) == False: #use function to check if a single characer is chiense
            return False 
    return True

#helper function, that find the help of a map from Traditional character in a string to simplified, attempts conversion of a string
def toSimplified(text, map):
    rettext = "" 
    #keep track of string index
    i = 0
    for char in str(text):
        #if the chracter is chiense 
        if contains_chinese(char) == True:
            #check if the character is in the map
            if char in map:
                rettext += map[char]
            else: #print that the character could not be converted
                print(f"trad char {char} cannot be converted based our map, using the same character as the original text")
                rettext += text[i]
        else: 
            rettext += text[i]
        i+=1#in this case we do not change =
    return rettext

#helper function, that find the help of our custom map a map from simplified characters to traditional, attempts conversion of a string
#handles cases where there are multiple mappings
def toTraditional(text, map):
    rettext = "" 
    #keep track of string index
    i = 0
    for char in str(text):
        #if the chracter is chiense 
        if contains_chinese(char) == True:
            #check if the character is in the map
            if char in map:
                #check the mapped list
                if len(map[char]) == 1:
                    rettext += map[char][0]
                else: 
                    print(f"simp char {char} has multiple possible traditional chars, use manual mapping, using the same character as the original text")
                    rettext += text[i] 
            else: #print that the character could not be converted
                print(f"simp char {char} cannot be converted based our map, using the same character as the original text")
                rettext += text[i]
        else: 
            rettext += text[i]
        i+=1#in this case we do not change =
    return rettext

#helper function determine if a character has diacritic or not
def contains_diacritic(char):
  nfkd_form = unicodedata.normalize('NFKD', char)  # Normalize to decompose characters
  for x in nfkd_form:
    if unicodedata.combining(x):  # Check if the character is a combining character
      return True
  return False


narrow down to words with more than 1 character:

In [ ]:
#load in data 
path = "main_final_list.csv" #"final_test_characters.csv"
test_data = pd.read_csv(path)
print(len(test_data))
test_data = test_data[test_data["word/character"].str.len() > 1]
print(len(test_data))

In [ ]:

test_data["test_definition"]=None
for index, row in test_data.iterrows(): 
    defi = str(row["definition"])      
    pro = str(row["pronunciation"])   #get the pronuciation. Replace all instances with _ for the test definition
    rettext = "" #set to return
    #keep track of string index
    i = 0
    for char in defi:
        #if the chracter is chinese, we skip (essenially skipping it)
        if contains_chinese(char) == True:
            continue
        else:
            rettext += char
        #for each substring we build, we see if we can find any  {} enclosed text, and these other combinations
        start = "{"
        end = "}"
        found_enclosed = getbetween(rettext, start,end) #returns "" 
        rettext=rettext.replace(start+found_enclosed+end, "") #works even if nothing is found

        start = "(Equivalent:"
        end = ")"
        found_enclosed = getbetween(rettext, start,end) #returns "" 
        rettext=rettext.replace(start+found_enclosed+end, "") #works even if nothing is found

        start = "(Traditional:"
        end = ")"
        found_enclosed = getbetween(rettext, start,end) #returns "" 
        rettext=rettext.replace(start+found_enclosed+end, "")

        start = "(Simplified:"
        end = ")"
        found_enclosed = getbetween(rettext, start,end) #returns "" 
        rettext=rettext.replace(start+found_enclosed+end, "")

        i+=1#in this case we do not change =
    rettext = rettext.replace(pro, "_")
    test_data.at[index, "test_definition"] = rettext.strip() #change definition

# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data = test_data.reindex()
test_data.to_csv("test_multi_list.csv", index=True)

Check for missing rows

In [ ]:
subset = test_data[test_data["test_definition"] == ""]
print(len(subset))
subset.head(n=19)

In [ ]:
test_data = test_data[test_data["test_definition"] != ""]
# Identify columns containing 'unnamed' (case-insensitive)
unnamed_cols = test_data.columns[test_data.columns.str.contains('unnamed', case=False)]
# Drop these columns
test_data.drop(columns=unnamed_cols, inplace=True)
test_data = test_data.reindex()
test_data.to_csv("test_multi_list.csv", index=True)
print(len(test_data))

### Part 7: Saving words (multi-character words) data to final json data

traditional data

In [ ]:
our_data = test_data[test_data["code"] == "t"]

json_index = our_data.to_json(orient='index')

# Print the JSON string
print(json_index)

simplified data

In [ ]:
our_data = test_data[test_data["code"] == "s"]

json_index = our_data.to_json(orient='index')

# Print the JSON string
print(json_index)